In [ ]:
"""
Code for data cleaning. 
Main goal is to find and create ONE 'one_name' for one company but with different variants of:

1. Company ID (sold_to_customer), 
2. Name spellings (sold_to_customer_n), 

each 'one_name' has to have eventually one definition for channel

1. Type (Direct or Indirect)
2. Type_with_channel (Direct - Direct or Indirect - Other or Indirect- Channel)
3. Company_type according to our specification (End user, Distributor, VAR, OEM, EPC etc.)
4. OPCO


"""

In [ ]:
"""
importing data from two files which were prepared accordingly from SAP system


order's columns:

['company_code_n', 'year_month', 'Date', 'FY', 'Sales group_case',
       'customer_group_code', 'Column1', 'BU Group OAC', 'BU Group_case',
       'bu2', 'bu_n', 'sales_person_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n', 'sold_to_region_n', 'eu_industry_n',
       'ec_eu_customer_n', 'eu_region_n', 'ec_eu_industry_n',
       'order_intake_amount_eur', 'OPCO', 'Type', 'Type with Channel',
       'Company time']
       
customer's columns

['Customer', 'Tier', 'Accounts', 'Company', 'Customer Name', 'Code',
       'Customer Group Name', 'Oscar i/d', 'Oscar Type', 'Indirect/Direct',
       'Channel3', 'New Type4', 'Correction_Indirect/Direct',
       'Correction_Channel', 'Correction_ Type', 'T&M', 'Partner page',
       'Tableau', 'Comments', 'Link']


"""

import pandas as pd
import numpy as np

order = pd.read_excel('data_files/full_order_data.xlsx',sheet_name='Data')
customer = pd.read_excel('data_files/full_customer_data.xlsx',sheet_name='data')

In [ ]:
"""
Cleaning old names. Step 1 - get a current SAP name of companies for Company ID (sold_to_customer)

1. Sort by FY (datetime), descending order
2. Creat 'company_id' list
3. check in cycle all lines of 'order' dataframe 
4. if Company ID (sold_to_customer) is in the list, then delete line  

"""

In [ ]:
#forting by date, descending order for keeping the newest
order_sorted_by_FY_date = order.sort_values(by='FY', ascending=False).reset_index(drop = True)
#it is necessary exlude all not essentional inforamtion:
order_sorted_by_FY_date = order_sorted_by_FY_date.loc[:,['FY','sold_to_customer', 'sold_to_customer_n', 'OPCO']]

In [ ]:
column_index = order_sorted_by_FY_date.columns.get_loc('sold_to_customer') #determine Company ID column index for iloc
company_id_exist_list = [] #creat 'company_id' list
result_df = order_sorted_by_FY_date.copy()

rng = len(order_sorted_by_FY_date)

for i in range(rng): #cycle for deleting repeting Company ID, saving the newest data
    
    if order_sorted_by_FY_date.iloc[i,column_index] not in company_id_exist_list:
        company_id_exist_list.append(order_sorted_by_FY_date.iloc[i,column_index])
    else:
        result_df = result_df.drop(i)
        print(i, rng, "Working : %.1f%% \r" % (i * 100 / rng), end='')

In [ ]:
"""
easier way to do the same thing
sevral seconds vs 30 minutes

"""
result2_df = order_sorted_by_FY_date.copy()
result2_df.drop_duplicates(subset='sold_to_customer', keep='first', inplace=True, ignore_index=True)

writer = pd.ExcelWriter('data_files/The_newest_names.xlsx', engine='xlsxwriter')
result_df.to_excel(writer, sheet_name='1')
result2_df.to_excel(writer, sheet_name='2')
writer.save()

In [ ]:
#to be continiued

In [ ]:
customer_uniq = customer.copy()
customer_uniq = customer_uniq.loc[:, ['Customer', 'Customer Name']]
merg_customer_uniq = customer_uniq.merge(result2_df, how='left', left_on='Customer', right_on='sold_to_customer')
merg_customer_uniq.dropna()
result2_df.iloc[11,2].upper().replace('-' , ' ').replace('"' , '').replace("," , " ").replace("." , " ").strip().split()
#pattern = '|'.join(['-','"', ',' ,'.'])
result2_df.dropna(inplace = True)
result2_df['sold_to_customer_n'] = result2_df['sold_to_customer_n'].str.replace('\-|\,|\.|\"' , " ", regex = True)
result2_df['sold_to_customer_n'] = result2_df['sold_to_customer_n'].str.upper().str.strip()  #.str.split()